# Explain the difficult algorithm 1

In [ ]:
import numpy as np

def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[ner_labels[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [ner_labels[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    all_metrics = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": all_metrics["overall_precision"],
        "recall": all_metrics["overall_recall"],
        "f1": all_metrics["overall_f1"],
        "accuracy": all_metrics["overall_accuracy"],
    }

O=0, PER=1, LOC=2, ORG=3

In [1]:
import numpy as np

# Etiket isimleri
ner_labels = ["O", "PER", "LOC", "ORG"]

# 2 örnek cümle var, her biri 5 token uzunluğunda
# logits: (batch_size=2, seq_len=5, num_labels=4)
logits = np.array([
    [  # 1. örnek
        [2.1, 0.3, 0.2, 0.1],   # token 1 -> "O"
        [0.1, 2.5, 0.2, 0.3],   # token 2 -> "PER"
        [0.1, 0.2, 2.0, 0.4],   # token 3 -> "LOC"
        [1.5, 0.1, 0.2, 0.1],   # token 4 -> "O"
        [0.1, 0.1, 0.2, 0.3],   # token 5 -> "ORG"
    ],
    [  # 2. örnek
        [0.2, 0.1, 2.3, 0.1],   # token 1 -> "LOC"
        [0.1, 2.0, 0.2, 0.2],   # token 2 -> "PER"
        [2.5, 0.1, 0.1, 0.2],   # token 3 -> "O"
        [0.3, 0.1, 0.2, 2.0],   # token 4 -> "ORG"
        [0.1, 0.1, 0.2, 0.3],   # token 5 -> "ORG"
    ]
])

# Gerçek etiketler (label id'leri)
# -100 = özel token (örn. [CLS], [PAD]) => ignore edilir
labels = np.array([
    [0, 1, 2, 0, -100],  # 1. örnek
    [2, 1, 0, 3, -100]   # 2. örnek
])

eval_preds = (logits, labels)


In [17]:
predictions = np.argmax(logits, axis=-1)

In [18]:
predictions 

array([[0, 1, 2, 0, 3],
       [2, 1, 0, 3, 3]])

In [14]:
true_labels_1 = [[ner_labels[l] for l in label if l != -100] for label in labels]
true_labels_1

[['O', 'PER', 'LOC', 'O'], ['LOC', 'PER', 'O', 'ORG']]

In [15]:
true_labels = []
for label in labels:
    print(label)
    labels_list= []
    for l in label:
        if l != -100:
            labels_list.append(ner_labels[l])

    true_labels.append(labels_list)
        

[   0    1    2    0 -100]
[   2    1    0    3 -100]


In [16]:
true_labels

[['O', 'PER', 'LOC', 'O'], ['LOC', 'PER', 'O', 'ORG']]

In [42]:
true_predictions = [
    [ner_labels[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_predictions

[['O', 'PER', 'LOC', 'O'], ['LOC', 'PER', 'O', 'ORG']]

In [44]:
true_p = []
for prediction, label in zip(predictions, labels):
    true_p_list = []
    for p,l in zip(prediction,label):
        if l!= -100:
            true_p_list.append(ner_labels[p])
    true_p.append(true_p_list)

true_p

[['O', 'PER', 'LOC', 'O'], ['LOC', 'PER', 'O', 'ORG']]

# Explain the difficult algorithm 2

In [45]:
# normalde Ankara güzeldir bizim veri setimizde şu şekilde yer alıyor: Ankara, Güzeldir, fakat model bunu şu şekilde tokenize ediyor: [CLS], "An", "##kara", "güzel", "##dir", [SEP]  gerçek etiketlerimiz de B-LOC,O olduğu için model hata verir tahmin yapmaz çünkü tokenized 6 oldu fakat etiket iki tane. Bu yüzden etiketleri tokenlere doğru bir şekilde yaymamız gerekiyor.

In [47]:
word_ids = [None, 0, 0, 1, 1, None] #tokenized icin donen ids listesi

In [ ]:
def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids: # iclerine bakiyrouz , 0 a geldik,
        if word_id != current_word: # ikisi de none o yuzden ilk adim olarak bunu atliyoruz. 0 none degil bu yuzden buraya girdik
            # Start of a new word!
            current_word = word_id
            label = -100 if word_id is None else labels[word_id] #labels'taki ilk etiket neyse onun strsini  aldik 
            new_labels.append(label)
        elif word_id is None: # buraya geldik ve special token oldugu icin -100 koyduk
            # Special token
            new_labels.append(-100)
        else: # ikinci 0 icin buraya girdik.
            # Same word as previous token
            label = labels[word_id] #yine index 0 olani aldik cunku ayni kelime fakat burda bu onceki kelimenin devami oldugu icin labels'taki etiket baslangic degil devam niteliginde bu yuzden onu I-...  yapmamiz lazim (tek sayi ise onu cift yap)
            # If the label is B-XXX we change it to I-XXX
            if label % 2 == 1:
                label += 1
            new_labels.append(label) #sonra yine str olarak al donguyu boylece devam ettir.

    return new_labels

sonuc :

[CLS]   An   ##kara  güzel  ##dir  [SEP]


 -100   1     2       0       0    -100

In [49]:
def tokenize_with_align_labels(example):
    tokenize  = tokenizer(example['tokens'], is_split_into_words=True)
    all_labels = example['ner_tags']
    new_labels = []
    deleted_row_count = 0
    for i, labels in enumerate(all_labels):
        word_inds = tokenize.word_ids(i)
        
        valid_word_inds = [w for w in word_inds if w is not None]
        if len(valid_word_inds) > 0 and max(valid_word_inds) >= len(labels): #some rows has labels and words not equal.
            print("WARNING: word_inds out of range!", word_inds, "labels len:", len(labels))
            new_labels.append([-100]*len(word_ids))
            deleted_row_count+=1
            continue
            
        new_labels.append(align_labels_with_tokens(labels,word_inds))


    tokenize['labels'] = new_labels
    return tokenize